In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import os
import re
import copy
import json

In [2]:
PIECES_TO_NUM = {
    "square": 0,
    "white_pawn": 1,
    "white_rook": 2,
    "white_knight": 3,
    "white_bishop": 4,
    "white_king": 5,
    "white_queen": 6,
    "white_unknown": 7,
    "black_pawn": -1,
    "black_rook": -2,
    "black_knight": -3,
    "black_bishop": -4,
    "black_king": -5,
    "black_queen": -6,
    "black_unknown": -7,
}

NUM_TO_PIECE = {
    0: "square",
    1: "white_pawn",
    2: "white_rook",
    3: "white_knight",
    4: "white_bishop",
    5: "white_king",
    6: "white_queen",
    7: "white_unknown",
    -1: "black_pawn",
    -2: "black_rook",
    -3: "black_knight",
    -4: "black_bishop",
    -5: "black_king",
    -6: "black_queen",
    -7: "black_unknown"
}

In [3]:
piece_images = {
    piece: Image.open(f'assets/{piece}.png') for piece in PIECES_TO_NUM.keys() if piece != "square"
}

In [4]:
game_state = np.array([
    [-2, -3, -4, -6, -5, -4, -3, -2],
    [-1, -1, -1, -1, -1, -1, -1, -1],
    [ 0,  0,  0,  0,  0,  0,  0,  -7],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  7],
    [ 1,  1,  1,  1,  1,  1,  1,  1],
    [ 2,  3,  4,  6,  5,  4,  3,  2],
])

def draw_chessboard(board, save=False):
    board = np.flipud(board)
    fig, ax = plt.subplots()

    chessboard_pattern = np.zeros((8, 8))
    chessboard_pattern[1::2, ::2] = 1
    chessboard_pattern[::2, 1::2] = 1

    ax.imshow(chessboard_pattern, cmap='gray', interpolation='none')

    for i in range(8):
        for j in range(8):
            piece_value = board[i, j]
            if piece_value != 0:
                piece_name = NUM_TO_PIECE[piece_value]
                piece_image = piece_images[piece_name]
                ax.imshow(piece_image, extent=[j - 0.5, j + 0.5, i - 0.5, i + 0.5])

    ax.set_xticks(np.arange(8))
    ax.set_yticks(np.arange(8))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks(np.arange(-0.5, 8, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, 8, 1), minor=True)
    ax.grid(which="minor", color="black", linestyle='-', linewidth=2)

    if save:
        fig.savefig(f"{save}.png")

    #plt.show()
    matplotlib.pyplot.close()
    
draw_chessboard(game_state)

In [5]:
def modify_game_state(game_state, move):
    new_game_state = copy.copy(np.flipud(game_state)) # A1 is 0,0
    if " -> " in move:
        init_pos = move.split(" -> ")[0]
        new_pos = move.split(" -> ")[1]

        init_y_pos = int(init_pos[-1]) - 1
        init_x_pos = int(ord(init_pos[-2]) - 96 - 1)

        new_y_pos = int(new_pos[-1]) - 1
        new_x_pos = int(ord(new_pos[-2]) - 96 - 1)      

        piece = new_game_state[init_y_pos, init_x_pos]
        new_game_state[init_y_pos, init_x_pos] = 0
        new_game_state[new_y_pos, new_x_pos] = piece

    elif " + " in move:
        init_pos = move.split(" + ")[0]
        new_pos = move.split(" + ")[1]

        init_y_pos = int(init_pos[-1]) - 1
        init_x_pos = int(ord(init_pos[-2]) - 96 - 1)

        new_y_pos = int(new_pos[-1]) - 1
        new_x_pos = int(ord(new_pos[-2]) - 96 - 1)      

        piece = new_game_state[init_y_pos, init_x_pos]
        new_game_state[init_y_pos, init_x_pos] = 0
        new_game_state[new_y_pos, new_x_pos] = piece

    elif " x " in move:
        init_pos = move.split(" x ")[0]
        new_pos = move.split(" x ")[1]

        init_y_pos = int(init_pos[-1]) - 1
        init_x_pos = int(ord(init_pos[-2]) - 96 - 1)

        new_y_pos = int(new_pos[-1]) - 1
        new_x_pos = int(ord(new_pos[-2]) - 96 - 1)      

        piece = new_game_state[init_y_pos, init_x_pos]
        new_game_state[init_y_pos, init_x_pos] = 0
        new_game_state[new_y_pos, new_x_pos] = piece

    elif "O-O-O" in move:
        if move[0] == "w":
            new_game_state[0, 2] = new_game_state[0, 4]
            new_game_state[0, 3] = new_game_state[0, 0]
            new_game_state[0, 0] = 0
            new_game_state[0, 4] = 0
        else:
            new_game_state[7, 2] = new_game_state[7, 4]
            new_game_state[7, 3] = new_game_state[7, 0]
            new_game_state[7, 0] = 0
            new_game_state[7, 4] = 0

    elif "O-O" in move:
        if move[0] == "w":
            new_game_state[0, 6] = new_game_state[0, 4]
            new_game_state[0, 5] = new_game_state[0, 7]
            new_game_state[0, 7] = 0
            new_game_state[0, 4] = 0
        else:
            new_game_state[7, 6] = new_game_state[7, 4]
            new_game_state[7, 5] = new_game_state[7, 7]
            new_game_state[7, 7] = 0
            new_game_state[7, 4] = 0
        

    return np.flipud(new_game_state)

In [6]:
# "->" is a move
# "x" is a capture
# "+" is check
# "#" is checkmate
# "O-O" is kingside castle
# "O-O-O" is queenside castle

game_fix = [
    "e2 -> e4",
    "e7 -> e6",
    "d2 -> d4",
    "d7 -> d5",
    "Nb1 -> Nd2",
    "Ng8 -> Nf6",
    "e4 -> e5",
    "Nf6 -> Nd7",
    "c2 -> c3",
    "c7 -> c5",
    "Bf1 -> Bd3",
    "Nb8 -> Nc6",
    "Ng1 -> Ne2",
    "c5 x d4",
    "c3 x d4",
    "f7 -> f6",
    "e5 x f6",
    "Nd7 x Nf6",
    "Nd2 -> Nf3",
    "Bf8 -> Bd6",
    "wO-O",
    "Qd8 -> Qc7",
    "Ne2 -> Nc3",
    "a7 -> a6",
    "Bc1 -> Bg5",
    "bO-O",
    "Bg5 -> Bh4",
    "Nf6 -> Nh5",
    "Rf1 -> Re1",
    "g7 -> g6",
    "Bh4 -> Bg5",
    "Qc7 -> Qg7",
    "Bg5 -> Be3",
    "Bc8 -> Bd7",
    "Bd3 -> Bf1",
    "Ra8 -> Re8",
    "g2 -> g3",
    "Re8 -> Re7",
    "Nf3 -> Ne5",
    "Nc6 x Ne5",
    "d4 x e5",
    "Bd6 x Be5",
    "Be3 -> Bc5",
    "Re7 -> Rf7",
    "Bc5 x Bf8",
    "Rf7 x Rf8",
    "Ra1 -> Rc1",
    "Be5 -> Bd4",
    "Re1 -> Re2",
    "Qg7 -> Qf6",
    "Qd1 -> Qd2",
    "Nh5 -> Ng7",
    "Bf1 -> Bg2",
    "Ng7 -> Nf5",
    "Kg1 -> Kh1",
    "Kg8 -> Kg7",
    "g3 -> g4",
    "Nf5 -> Nh4",
    "g4 -> g5",
    "Qf6 -> Qf4",
    "Nc3 x Nd5",
    "Qf4 -> Qg4",
    "Nd5 -> Ne3",
    "Bd4 x Be3",
    "f2 x e3",
    "Bd7 -> Bb5",
    "Re2 -> Rf2",
    "Nh4 -> Nf5",
    "Rc1 + Rc7",
    "Kg7 -> Kg8",
    "e3 -> e4"
]

game_moving = [
    "e2 -> e4",
    "c7 -> c5",
    "Ng1 -> Nf3",
    "e7 -> e6",
    "Nb1 -> Nc3",
    "Nb8 -> Nc6",
    "d2 -> d4",
    "c5 x d4",
    "Nf3 x Nd4",
    "a7 -> a6",
    "f2 -> f4",
    "Qd8 -> Qc7",
    "Bc1 -> Be3",
    "b7 -> b5",
    "Bf1 -> Bd3",
    "Bc8 -> Bb7",
    "Qd1 -> Qf3",
    "Ng8 -> Nf6",
    "Nd4 -> Nb3",
    "Nc6 -> Nb4",
    "a2 -> a3",
    "Nb4 x Nd3",
    "c2 x d3",
    "d7 -> d6",
    "Ra1 -> Rc1",
    "Qc7 -> Qd8",
    "f4 -> f5",
    "Bf8 -> Be7",
    "f5 x e6",
    "f7 x e6",
    "Nb3 -> Nd4",
    "Qd8 -> Qd7",
    "Qf3 -> Qh3",
    "Bb7 -> Bc8",
    "wO-O",
    "bO-O",
    "Rf1 -> Rf3",
    "Be7 -> Bd8",
    "Rc1 -> Rf1",
    "Bd8 -> Bb6",
    "Qh3 -> Qg3",
    "Qd7 -> Qe7",
    "Kg1 -> Kh1",
    "Bc8 -> Bb7",
    "Nc3 -> Ne2",
    "Bb6 x Bd4",
    "Ne2 x Nd4",
    "Rf8 -> Rf7",
    "Qg3 -> Qh3",
    "Ra8 -> Re8",
    "Be3 -> Bg5",
    "Bb7 -> Bc8",
    "Bg5 x Bf6",
    "Rf7 x Rf6",
    "Rf3 x Rf6",
    "g7 x f6",
    "Qh3 + Qg3",
    "Kg8 -> Kh8",
    "Nd4 -> Nc6",
    "Qe7 -> Qc7",
    "Qg3 -> Qh4",
    "Qc7 x Qc6",
    "Qh4 x Qf6",
    "Kh8 -> Kg8",
    "Qf6 + Qg5",
    "Kg8 -> Kh8",
    "Rf1 -> Rf7"
]

frames_fix = {
    1: 60,
    2: 110,
    3: 185,
    4: 240,
    5: 360,
    6: 470,
    7: 531,
    8: 660,
    9: 765,
    10: 878,
    11: 1005,
    12: 1153,
    13: 1232,
    14: 1365,
    15: 1555,
    16: 1711,
    17: 1825,
    18: 1968,
    19: 2113,
    20: 2202,
    21: 2345,
    22: 2470,
    23: 2640,
    24: 2740,
    25: 2882,
    26: 2996,
    27: 3125,
    28: 3204,
    29: 3330,
    30: 3404,
    31: 3542,
    32: 3710,
    33: 3772,
    34: 3846,
    35: 3982,
    36: 4025,
    37: 4140,
    38: 4206,
    39: 4278,
    40: 4448,
    41: 4544,
    42: 4687,
    43: 4789,
    44: 4850,
    45: 5030,
    46: 5124,
    47: 5237,
    48: 5314,
    49: 5396,
    50: 5462,
    51: 5560,
    52: 5635,
    53: 5744,
    54: 5880,
    55: 5969,
    56: 6030,
    57: 6111,
    58: 6215,
    59: 6307,
    60: 6378,
    61: 6613,
    62: 6700,
    63: 6797,
    64: 6866,
    65: 6954,
    66: 7100,
    67: 7164,
    68: 7283,
    69: 7374,
    70: 7448,
    71: 7552
}

frames_moving = {
    1: 55,
    2: 150,
    3: 289,
    4: 373,
    5: 520,
    6: 625,
    7: 757,
    8: 853,
    9: 964,
    10: 1058,
    11: 1149,
    12: 1227,
    13: 1360,
    14: 1427,
    15: 1544,
    16: 1596,
    17: 1731,
    18: 1842,
    19: 1956,
    20: 2090,
    21: 2158,
    22: 2330,
    23: 2436,
    24: 2552,
    25: 2605,
    26: 2686,
    27: 2797,
    28: 2890,
    29: 3100,
    30: 3358,
    31: 3460,
    32: 3525,
    33: 3626,
    34: 3717,
    35: 3864,
    36: 3966,
    37: 4070,
    38: 4139,
    39: 4272,
    40: 4400,
    41: 4475,
    42: 4551,
    43: 4610,
    44: 4739,
    45: 4828,
    46: 5034,
    47: 5282,
    48: 5397,
    49: 5500,
    50: 5582,
    51: 5739,
    52: 5876,
    53: 6047,
    54: 6196,
    55: 5420,
    56: 6603,
    57: 6691,
    58: 6794,
    59: 6915,
    60: 7036,
    61: 7116,
    62: 7225,
    63: 7437,
    64: 7529,
    65: 7644,
    66: 7693,
    67: 7845
}

In [7]:
data = {} # data contains the list of the move and the game_state + frame_number after each move
data["moves"] = game_fix
data["game_states"] = []

In [8]:
initial_game_state = np.array([
    [-2, -3, -4, -6, -5, -4, -3, -2],
    [-1, -1, -1, -1, -1, -1, -1, -1],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 1,  1,  1,  1,  1,  1,  1,  1],
    [ 2,  3,  4,  6,  5,  4,  3,  2],
])

# We always 
data["game_states"].append({"frame": 0,
                            "gs": initial_game_state.tolist()
                           })

draw_chessboard(initial_game_state, "step1/games/fix/gs_0")

new_gs = modify_game_state(initial_game_state, game_fix[0])

# In you case, the frame number won't be 1, this is just an example here.
data["game_states"].append({"frame": frames_fix.get(1),
                            "gs": new_gs.tolist()
                           })
draw_chessboard(new_gs, "step1/games/fix/gs_1")

i = 2
for j, g in enumerate(game_fix[1:]):
    new_gs = modify_game_state(new_gs, g)
    data["game_states"].append({"frame": frames_fix.get(i),
                            "gs": new_gs.tolist()
                           })
    draw_chessboard(new_gs, f"step1/games/fix/gs_{i}")
    i = i + 1

In [9]:
# save the data in a json file
json.dump(data, open("step1/games/game_fix.json", "w"))

In [10]:
initial_game_state[0, :] = -7
initial_game_state[1, :] = -7

initial_game_state[6, :] = 7
initial_game_state[7, :] = 7

draw_chessboard(initial_game_state, "step1/games/fix/unknown/gs_0")

new_gs = modify_game_state(initial_game_state, game_fix[0])
draw_chessboard(new_gs, "step1/games/fix/unknown/gs_1")

i = 2
for j, g in enumerate(game_fix[1:]):
    new_gs = modify_game_state(new_gs, g)
    draw_chessboard(new_gs, f"step1/games/fix/unknown//gs_{i}")
    i = i + 1

In [11]:
data = {} # data contains the list of the move and the game_state + frame_number after each move
data["moves"] = game_moving
data["game_states"] = []

In [12]:
initial_game_state = np.array([
    [-2, -3, -4, -6, -5, -4, -3, -2],
    [-1, -1, -1, -1, -1, -1, -1, -1],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 1,  1,  1,  1,  1,  1,  1,  1],
    [ 2,  3,  4,  6,  5,  4,  3,  2],
])

# We always 
data["game_states"].append({"frame": 0,
                            "gs": initial_game_state.tolist()
                           })

draw_chessboard(initial_game_state, "step1/games/moving/gs_0")

new_gs = modify_game_state(initial_game_state, game_moving[0])

# In you case, the frame number won't be 1, this is just an example here.
data["game_states"].append({"frame": frames_moving.get(1),
                            "gs": new_gs.tolist()
                           })
draw_chessboard(new_gs, "step1/games/moving/gs_1")

i = 2
for j, g in enumerate(game_moving[1:]):
    new_gs = modify_game_state(new_gs, g)
    data["game_states"].append({"frame": frames_moving.get(i),
                            "gs": new_gs.tolist()
                           })
    draw_chessboard(new_gs, f"step1/games/moving/gs_{i}")
    i = i + 1

In [13]:
# save the data in a json file
json.dump(data, open("step1/games/game_moving.json", "w"))

In [14]:
initial_game_state[0, :] = -7
initial_game_state[1, :] = -7

initial_game_state[6, :] = 7
initial_game_state[7, :] = 7

draw_chessboard(initial_game_state, "step1/games/moving/unknown/gs_0")

new_gs = modify_game_state(initial_game_state, game_moving[0])
draw_chessboard(new_gs, "step1/games/moving/unknown/gs_1")

i = 2
for j,g in enumerate(game_moving[1:]):
    new_gs = modify_game_state(new_gs, g)
    draw_chessboard(new_gs, f"step1/games/moving/unknown//gs_{i}")
    i = i + 1

In [15]:
# You can retrieve the game state from the json file
# but don't forget to convert the list to a numpy array
draw_chessboard(np.array(data["game_states"][0]["gs"]))